# HW2

In [1]:
import sys
import scipy.optimize as opt
import math
import numpy as np
import pandas as pd
import pathlib
import scipy.linalg as linalg
import multiprocessing as mp
from collections import OrderedDict
np.set_printoptions(precision=10)
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=1000)
pd.set_option("display.width", 10)
pd.set_option("display.max_rows", 10)

## Model

The data is generated by the following indirect utility function:

$$
u_{ijm} = \beta_0 + x_{1j} (\beta_1 + \beta_1^u \nu_{i1}) + \xi_j + \epsilon_{ijm}
$$

where $\nu_{i1} \sim N(0, 1)$ and $\epsilon_{ij}$ follows the Type I extreme distribution. Assume $E[\xi_j \mid x_{1j}] = 0$.

In [2]:
path = "Data2019PS2.csv"
dtypes = OrderedDict([
    ("market_id", int), 
    ("product_id", int), 
    ("market_sales", float), 
])
df = pd.read_csv(path, header=None, names=dtypes.keys(), dtype=dtypes)
total_populations = [1000000, 2000000, 4000000]

for p in total_populations:
    df["market_share_{}".format(p)] = df["market_sales"] / p

product_chars = pd.DataFrame({
    "product_id": list(range(1, 11)), 
    "char_1": np.arange(0.01, 0.11, 0.01), 
})

df = pd.merge(df, product_chars, on="product_id")
df

,market_id,product_id,market_sales,market_share_1000000,market_share_2000000,market_share_4000000,char_1
0,1,1,127379.0,0.127379,0.063689,0.031845,0.01
1,2,1,111019.0,0.111019,0.055510,0.027755,0.01
2,3,1,124636.0,0.124636,0.062318,0.031159,0.01
3,5,1,123446.0,0.123446,0.061723,0.030862,0.01
4,6,1,213350.0,0.213350,0.106675,0.053338,0.01
...,...,...,...,...,...,...,...
7053,990,10,106077.0,0.106077,0.053039,0.026519,0.10
7054,991,10,149099.0,0.149099,0.074550,0.037275,0.10
7055,995,10,129707.0,0.129707,0.064853,0.032427,0.10
7056,998,10,146353.0,0.146353,0.073177,0.036588,0.10


## (a) Estimate a plain logit model, assuming that $β^u_1 = 0$

We can estimate the logit model by simply regressing:

$$
\log(s_{jm}) - \log(s_{0m}) = \beta_0 + \beta_1 x_{1j} + \epsilon_{ijm}
$$

where $s_{jm}$ is the market share of the product $j$ in the market $m$. $s_{0m}$ is the market share of the outside option.

In [3]:
market_size = 1000
outside_option_shares = df.groupby("market_id")[
    "market_share_1000000"].apply(lambda d: 1.0 - np.sum(d))
outside_option_shares = outside_option_shares.reset_index()
outside_option_shares.columns = ["market_id", "outside_option_1000000"]
df = pd.merge(
    df, outside_option_shares, on="market_id"
).sort_values(["market_id", "product_id"])
df

,market_id,product_id,market_sales,market_share_1000000,market_share_2000000,market_share_4000000,char_1,outside_option_1000000
0,1,1,127379.0,0.127379,0.063689,0.031845,0.01,0.045863
1,1,2,128477.0,0.128477,0.064239,0.032119,0.02,0.045863
2,1,3,132527.0,0.132527,0.066264,0.033132,0.03,0.045863
3,1,4,136916.0,0.136916,0.068458,0.034229,0.04,0.045863
4,1,6,139476.0,0.139476,0.069738,0.034869,0.06,0.045863
...,...,...,...,...,...,...,...,...
6629,1000,4,135554.0,0.135554,0.067777,0.033889,0.04,0.045303
6633,1000,5,135495.0,0.135495,0.067748,0.033874,0.05,0.045303
6630,1000,6,138286.0,0.138286,0.069143,0.034571,0.06,0.045303
6631,1000,7,140472.0,0.140472,0.070236,0.035118,0.07,0.045303


In [4]:
y = (np.log(df["market_share_1000000"]) \
     - np.log(df["outside_option_1000000"])).values
X = np.ones([df.shape[0], 2])
X[:, 1] = df["char_1"]
beta = np.linalg.solve(np.transpose(X)@X, np.transpose(X)@y)
beta

array([0.9983356155, 1.9726098087])

We have $(\hat{\beta}^{\text{logit}}_0, \hat{\beta}^{\text{logit}}_1) = (0.9983356155, 1.9726098087)$.

In [5]:
res = y - X @ beta
cov_mat = np.sum(np.power(res, 2)) * np.linalg.inv(np.transpose(X) @ X) \
    / (X.shape[0] - X.shape[1])
cov_mat

array([[ 0.0000000518, -0.000000742 ],
       [-0.000000742 ,  0.0000135967]])

The Var-Cov matrix is 

$$
Var((\hat{\beta}^{\text{logit}}_0, \hat{\beta}^{\text{logit}}_1)) = \left(
    \begin{array}{cc}
      0.0000000518 & -0.000000742 \\
      -0.000000742 & 0.0000135967
    \end{array}
  \right)
$$

## (b)

In order to see the effects of population-potential buyers for this product, redo
(a) with $N = 2000000$ and $N = 4000000$. Discuss which parameter would be biased and explain why this is the case.

In [6]:
for p in total_populations[1:]:
    outside_option_shares = df.groupby("market_id")[
        "market_share_{}".format(p)].apply(lambda d: 1.0 - np.sum(d))
    outside_option_shares = outside_option_shares.reset_index()
    outside_option_shares.columns = ["market_id", "outside_option_{}".format(p)]
    df = pd.merge(df, outside_option_shares, on="market_id")
    y = (np.log(df["market_share_{}".format(p)]) \
        - np.log(df["outside_option_{}".format(p)])).values
    X = np.ones([df.shape[0], 2])
    X[:, 1] = df["char_1"]
    print(np.linalg.solve(np.transpose(X)@X, np.transpose(X)@y))
    res = y - X @ beta
    cov_mat = np.sum(np.power(res, 2)) * np.linalg.inv(np.transpose(X) @ X) \
        / (X.shape[0] - X.shape[1])
    print(cov_mat)

[-2.1695925239  1.8575547123]
[[ 0.0065763923 -0.0942519001]
 [-0.0942519001  1.7271521213]]
[-3.2391413402  1.8535079949]
[[ 0.0117393421 -0.1682465465]
 [-0.1682465465  3.0830930662]]


When each market size = 2 million, $(\hat{\beta}^{\text{logit}}_0, \hat{\beta}^{\text{logit}}_1) = (-2.1695925239, 1.8575547123)$ and

  $$
    Var((\hat{\beta}^{\text{logit}}_0, \hat{\beta}^{\text{logit}}_1)) = \left(
        \begin{array}{cc}
          0.0065763923 & -0.0942519001 \\
          -0.0942519001 & 1.7271521213
        \end{array}
      \right)
  $$

When each market size = 4 million, $(\hat{\beta}^{\text{logit}}_0, \hat{\beta}^{\text{logit}}_1) = (-3.2391413402, 1.8535079949)$ and

  $$
    Var((\hat{\beta}^{\text{logit}}_0, \hat{\beta}^{\text{logit}}_1)) = \left(
        \begin{array}{cc}
          0.0117393421 & -0.1682465465 \\
          -0.1682465465 & 3.0830930662
        \end{array}
      \right)
  $$

$\beta_0$ is sensitive to the market size while $\beta_1$ is robust.

## (c)

1. First compute $\delta^*$.
1. Obtain $\xi^*$ as a OLS residual.
1. Search the random coefficient $\beta^u_1$ via MSM.

In [7]:
class BLP(object):
    """
    df should include
        - market_id
        - product_id
        - market_share
        - outside_option_share
    """
    def __init__(self, blp_df, product_chars, char_cols, num_simulations, random_seed=None):
        self.df = blp_df
        self.product_chars = product_chars
        self.char_cols = char_cols
        if random_seed is None:
            random_seed = 1
        
        self.random_state = np.random.RandomState(seed=random_seed)
        self.num_simulations = num_simulations
        self.num_chars = len(char_cols)
        self.num_markets = len(np.unique(df["market_id"]))
        self.num_products = len(np.unique(df["product_id"]))
        self._get_missing_market_product_pairs()
        self._prepare_BLP()
        
        
    def _get_missing_market_product_pairs(self):
        # Compute product_id that does not exist in each market
        temp_df = pd.DataFrame({
            "market_id": np.repeat(range(1, self.num_markets+1), self.num_products), 
            "product_id": np.tile(range(1, self.num_products+1), self.num_markets), 
        })
        temp_df = pd.merge(self.df, temp_df, on=["market_id", "product_id"], how="outer")

        # Existing / missing (market_id, product_id) pair
        existing_market_product_indices = temp_df.loc[
            ~np.isnan(temp_df["market_sales"]), ["market_id", "product_id"]].values - 1
        missing_market_product_indices = temp_df.loc[
            np.isnan(temp_df["market_sales"]), ["market_id", "product_id"]].values - 1

        self.existing_market_product_indices = existing_market_product_indices[
            np.lexsort(
                (existing_market_product_indices[:, 1], existing_market_product_indices[:, 0]))
        ]
        self.missing_market_product_indices = missing_market_product_indices[
            np.lexsort(
                (missing_market_product_indices[:, 1], missing_market_product_indices[:, 0]))
        ]
        
        
    def _prepare_BLP(self):
        self.delta_init = np.zeros(shape=[self.num_markets, self.num_products])
        self.delta_init[
            self.existing_market_product_indices[:, 0], 
            self.existing_market_product_indices[:, 1]
        ] = np.log(self.df["market_share"]) - np.log(self.df["outside_option_share"])
        
        self.log_data_market_share = np.zeros(shape=[self.num_markets, self.num_products])
        self.log_data_market_share[
            self.existing_market_product_indices[:, 0], 
            self.existing_market_product_indices[:, 1]
        ] = np.log(self.df["market_share"])
        
        self.X = np.ones([self.df.shape[0], 2])
        self.X[:, 1:self.num_chars+1] = self.df[self.char_cols]
        self.X_t = np.transpose(self.X)
        self.X_lu = linalg.lu_factor(self.X_t @ self.X)
        
        
    def get_BLP_estimate(self):
        noise_matrices = [self.random_state.standard_normal(
            size=(self.num_simulations, self.num_chars))]
        
        results = self._get_BLP_estimate((
            self.num_markets, 
            noise_matrices,  
            self.product_chars, 
            self.char_cols, 
            self.existing_market_product_indices, 
            self.missing_market_product_indices, 
            self.log_data_market_share, 
            self.delta_init, 
            self.X, 
            self.X_t, 
            self.X_lu, 
            False
        ))
        return results[0]
    
    
    @staticmethod
    def _get_BLP_estimate(args):
        num_markets, \
        noise_matrices, \
        product_chars, \
        char_cols, \
        existing_market_product_indices, \
        missing_market_product_indices, \
        log_data_market_share, \
        delta_init, \
        X, \
        X_t, \
        X_lu, \
        retun_only_coefs = args
        
        results = []
        for noise_matrix in noise_matrices:
            base_mu_matrix = noise_matrix @ product_chars.loc[:, char_cols].values.reshape(1, 10)
            mu_matrices_base = np.repeat(base_mu_matrix[:, None, :], num_markets, axis=1)
            mu_matrices_base[
                :, 
                missing_market_product_indices[:, 0], 
                missing_market_product_indices[:, 1]
            ] = -np.inf
            finite_ind = (mu_matrices_base != -np.inf)
            result = opt.minimize_scalar(
                BLP.BLP_obj_func, 
                tol=1e-6, 
                args=(
                    mu_matrices_base, 
                    finite_ind, 
                    log_data_market_share, 
                    delta_init, 
                    existing_market_product_indices, 
                    X, 
                    X_t, 
                    X_lu
                )
            )

            _, beta_bar = BLP.msm_inner(
                result.x, 
                mu_matrices_base, 
                finite_ind, 
                log_data_market_share, 
                delta_init, 
                existing_market_product_indices, 
                X, 
                X_t, 
                X_lu
            )
            
            results.append([result, beta_bar])
        
        if retun_only_coefs:
            coef_list = np.empty([len(noise_matrices), 3])
            for i, r in enumerate(results):
                coef_list[i, 0] = r[0].x
                coef_list[i, 1:3] = r[1]
                
            return coef_list
            
        return results
    
    
    @staticmethod
    def BLP_obj_func( 
        beta_u, 
        mu_matrices_base, 
        finite_ind, 
        log_data_market_share, 
        delta_init, 
        existing_market_product_indices, 
        X, 
        X_t, 
        X_lu 
    ):
        y, beta_bar = BLP.msm_inner(
            beta_u, 
            mu_matrices_base, 
            finite_ind, 
            log_data_market_share, 
            delta_init, 
            existing_market_product_indices, 
            X, 
            X_t, 
            X_lu
        )
        xi = y - X @ beta_bar
        return np.sum(np.power(xi, 2))
    
    
    # outer loop
    @staticmethod
    def msm_inner(
        beta_u, 
        mu_matrices_base, 
        finite_ind, 
        log_data_market_share, 
        delta_init, 
        existing_market_product_indices, 
        X, 
        X_t, 
        X_lu
    ):
        exp_mu_matrices = np.copy(mu_matrices_base)
        exp_mu_matrices[finite_ind] *= beta_u
        exp_mu_matrices = np.exp(exp_mu_matrices)
        delta_converged = BLP.compute_fixed_point(
            exp_mu_matrices, 
            log_data_market_share, 
            delta_init
        )
        y = delta_converged[
            existing_market_product_indices[:, 0], 
            existing_market_product_indices[:, 1]
        ]
        beta_bar = linalg.lu_solve(X_lu, X_t@y)
        return y, beta_bar
    
    
    # inner loop
    def compute_fixed_point(
        exp_mu_matrices, 
        log_data_market_share, 
        delta_init, 
        eps=1e-6, 
        max_iter=1000
        ):
        delta = delta_init
        for i in range(max_iter):
            choice_prob_matrix = np.exp(delta)[None, :, :] * exp_mu_matrices
            choice_prob_matrix = choice_prob_matrix / (
                1 + np.sum(choice_prob_matrix, axis=2))[:, :, None]
            simulated_market_share = np.mean(choice_prob_matrix, axis=0)
            if i == 0:
                ind = (simulated_market_share > 0)

            log_simulated_market_share = simulated_market_share
            log_simulated_market_share[ind] = np.log(simulated_market_share[ind])

            delta_new = delta + log_data_market_share - log_simulated_market_share
            if np.sum(np.power(delta_new - delta, 2)) < eps:
                break

            delta = delta_new

        return delta
    

In [8]:
blp_df = df.loc[
    :, [
        "market_id", 
        "product_id", 
        "market_sales", 
        "market_share_1000000", 
        "outside_option_1000000", 
        "char_1"]
].copy()
blp_df.rename(columns={
    "market_share_1000000": "market_share", 
    "outside_option_1000000": "outside_option_share"
}, inplace=True)
num_simulations = 1000
random_seed = 1

blp_instance = BLP(blp_df, product_chars, ["char_1"], num_simulations, random_seed)

In [9]:
blp_instance.get_BLP_estimate()

[     fun: 0.5585382576164789
     nfev: 15
      nit: 11
  success: True
        x: 1.086891774976817, array([0.99973109  , 1.9318972474])]

In [10]:
result, beta_bar = blp_instance.get_BLP_estimate()

In [11]:
result

     fun: 0.5595509091229924
    nfev: 31
     nit: 27
 success: True
       x: 0.9112589339845125

In [12]:
beta_bar

array([0.9994729234, 1.9485169337])

The estimation result is
$$
(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}^u_1) = (0.999731, 1.931897, 0.5585383).
$$

## (d)

Use the first 10 and 100 markets to estimate the model. The result:

When the market size is 10, 
$$
(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}^u_1) = (0.997, 1.960, 0.3038002).
$$


When the market size is 100, 
$$
(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}^u_1) = (0.999, 1.947, 0.6733046).
$$

The random coefficient highly depends on the size of the market used, while other 2 parameters do not vary so much.

In [13]:
num_simulations = 1000
random_seed = 1
blp_df_10 = blp_df.loc[blp_df["market_id"] <= 10, :]
blp_df_100 = blp_df.loc[blp_df["market_id"] <= 100, :]
blp_instance_10 = BLP(blp_df_10, product_chars, ["char_1"], num_simulations, random_seed)
blp_instance_100 = BLP(blp_df_100, product_chars, ["char_1"], num_simulations, random_seed)

In [14]:
result_10, beta_bar_10 = blp_instance_10.get_BLP_estimate()

In [15]:
result_10

     fun: 0.0051985218869492755
    nfev: 25
     nit: 21
 success: True
       x: 0.3038002163412557

In [16]:
beta_bar_10

array([0.9970700475, 1.9596778777])

In [17]:
result_100, beta_bar_100 = blp_instance_100.get_BLP_estimate()

In [18]:
result_100

     fun: 0.05743458643525041
    nfev: 31
     nit: 27
 success: True
       x: 0.6733046149497918

In [19]:
beta_bar_100

array([0.9992604407, 1.947279725 ])